DataSet : https://www.kaggle.com/c/digit-recognizer/

Below are the couple things to address the flaws mentioned in experimential design

1. All the test and training dataset is scaled to ensure the data is centered with a unit deviation before employing PCA. - 332 components
2. After implementing PCA, the RF classifier is again trained with the new reduced train and test data and are searched for new hypertuning parameters.
3. As a modification to the design, the model is trained to fit alone on the train data first with .095 varience and then used PCA.transform to fit the test data. The results however did not yield better performance

In [46]:
import numpy as  np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [47]:
X = pd.read_csv("/content/train.csv")
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [48]:
X.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
X['label'].nunique()
y = X['label']
X = X.iloc[:,1:]

In [50]:
#Set aside 20% of the data for test
split = len(X) - int((len(X)*20/100))
X_train = X.iloc[0:split,:]
X_test = X.iloc[split:,:]
y_train = y.iloc[0:split]
y_test = y.iloc[split:]

In [51]:
rf = rfc(random_state=1234,n_jobs=-1, n_estimators=90)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=-1,
                       oob_score=False, random_state=1234, verbose=0,
                       warm_start=False)

In [52]:
rf.score(X_test,y_test)

0.9663095238095238

In [53]:
print(rf.get_params());

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 90, 'n_jobs': -1, 'oob_score': False, 'random_state': 1234, 'verbose': 0, 'warm_start': False}


Set Up the Search Grid


In [54]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = {'n_estimators':[5,25,125,150,200,250],
                  'bootstrap':[False, True],
                  'min_samples_split':[2,4,8,16,None],
                  'max_depth':[10,100,1000,None],
                  'criterion':['entropy','gini'],
                  'max_features' : ['log2', 'sqrt']}

In [13]:
rs_random = RandomizedSearchCV(estimator = rf, 
                        param_distributions = param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 2, 
                        n_jobs=-1, 
                        random_state=42)

rs_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 34.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 55.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [14]:
rs_random.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_split': 2,
 'n_estimators': 200}

Build the Model - Before PCA, After Search Grid

In [15]:
from datetime import datetime
from sklearn.metrics import f1_score as f1

rf = rfc(n_estimators=200,criterion='gini',bootstrap=False,max_depth=100,max_features='sqrt',min_samples_split=2)
start=datetime.now()
rf.fit(X_train,y_train)
end=datetime.now()
print("Time:", end-start)
print("Accuracy",rf.score(X_test,y_test))
y_pred = rf.predict(X_test)
print("F1 Accuracy", f1(y_pred,y_test,average='macro'))

Time: 0:01:16.938201
Accuracy 0.9714285714285714
F1 Accuracy 0.9711720033721896


In [16]:
#Fit the entire model - Kaggle Submission
start=datetime.now()
rf.fit(X,y)
end=datetime.now()
print("Time:", end-start)
test = pd.read_csv("/content/test.csv")
y_before_pca_pred = rf.predict(test)
df = pd.DataFrame({'ImageId': test.index + 1, 'Label': y_before_pca_pred})
df.to_csv('BeforePCA.csv', index = False)

Time: 0:01:35.034722


Implement PCA on entire train.csv

 Step1 : Scale the Data and then do the fit

In [55]:
from sklearn.decomposition import PCA
scalar = StandardScaler()
X = np.concatenate((X_train,X_test), axis=0)
X = StandardScaler().fit_transform(X)

In [56]:
pca = PCA(.95)
pca.fit(X)
X = pca.transform(X)
pca.n_components_

320

In [57]:
X.shape

(42000, 320)

In [58]:
#Split back X to test and train 
X = X.astype(int)
X_train = X[0:split,:]
X_test = X[split:,:]

In [23]:
#Set up the search grid after employing PCA

rs_random_with_pca = RandomizedSearchCV(estimator = rf, 
                        param_distributions = param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 2, 
                        n_jobs=-1, 
                        random_state=42)

rs_random_with_pca.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 31.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=False,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=100,
                                                    max_features='sqrt',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [24]:
rs_random.best_params_


{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_split': 2,
 'n_estimators': 250}

In [59]:
#Train the RF model first on the train daa set, evaluate the F1 Score and then implement it on train.csv
rf_after_pca = rfc(n_estimators=250,criterion='entropy',bootstrap=False,max_depth=None,max_features='sqrt',min_samples_split=2)
start=datetime.now()
rf_after_pca.fit(X_train,y_train)
end = datetime.now()
print(end-start)
print("Accuracy :",rf_after_pca.score(X_test,y_test))
y_pred_pca = rf_after_pca.predict(X_test)
print("F1 Accuracy: ", f1(y_pred_pca,y_test,average='macro'))

0:00:58.939490
Accuracy : 0.9432142857142857
F1 Accuracy:  0.9422577876655284


In [60]:
#Apply PCA on test dataset
test_pca = pd.read_csv("/content/test.csv")
test_pca = pca.transform(test_pca)
pca.n_components_


320

In [61]:
#Train the model on completedata and submit to Kaggle
start=datetime.now()
rf_after_pca.fit(X,y)
end=datetime.now()
print("Time:", end-start)


Time: 0:01:14.213400


In [64]:
test_pca = test_pca.astype(int)
y_after_pca_pred = rf_after_pca.predict(test_pca)
df = pd.DataFrame({'ImageId': test.index + 1, 'Label': y_after_pca_pred})
df.to_csv('AfterPCA.csv', index = False)

In [102]:
#Combined Training DataSet

X1 = pd.read_csv("/content/train.csv")
X2 = pd.read_csv("/content/test.csv")


In [103]:
len(X1)

42000

In [104]:
df = pd.concat([X1,X2],axis=0)
df_test = df['label']
df_train = df.iloc[:,1:]


In [105]:
df_test

0        1.0
1        0.0
2        1.0
3        4.0
4        0.0
        ... 
27995    NaN
27996    NaN
27997    NaN
27998    NaN
27999    NaN
Name: label, Length: 70000, dtype: float64

In [106]:
df_train = StandardScaler().fit_transform(df_train)


In [90]:
pca = PCA(.95)
pca.fit(df_train)
df_test = pca.transform(df_train)
pca.n_components_

332

In [109]:
df_actual_train = df_train[0:42000,:]
df_actual_test = df_train[42000:,:]
df_actual_label = df_test[df_test.notnull()]
df_actual_pred = df_test[df_test.isnull()]

In [112]:
df_actual_label

0        1.0
1        0.0
2        1.0
3        4.0
4        0.0
        ... 
41995    0.0
41996    1.0
41997    7.0
41998    6.0
41999    9.0
Name: label, Length: 42000, dtype: float64

In [ ]:
rf_after_pca = rfc(n_estimators=250,criterion='entropy',bootstrap=False,max_depth=None,max_features='sqrt',min_samples_split=2)
start=datetime.now()
rf_after_pca.fit(df_actual_train,df_actual_label)
end = datetime.now()
print(end-start)

In [111]:

y_after_pca_pred1 = rf_after_pca.predict(df_actual_test)
df = pd.DataFrame({'ImageId': test.index + 1, 'Label': y_after_pca_pred1})
df.to_csv('AfterPCA.csv', index = False)